### Autograd 사용
- 어떤 tensor가 학습에 필요한 tensor라면 `backpropagation`을 통해 gradient를 구해야 함

- `tensor`의 gradient를 구하기 위한 조건

    - `requires_grad = True` 로 설정되어 있어야함, default는 False
    
    - `backpropagation`을 시작하는 지점의 output은 `scalar` 형태   
    
- `tensor`의 gradient를 계산하기 위해서는 output 지점의 tensor에서 `.backward()` 함수를 호출하면 됨

- gradient 값을 확인하려면 `requires_grad=True`로 생성한 tensor에 `.grad` 를 통해 확인할 수 있음

### Autograd 개념

`x`는 2 x 2 크기의 1로 구성된 tensor를 생성한 것이고

`y`는 `requires_grad=True`로 설정하여 역전파 과정을 수행할 수 있게 됨

In [1]:
import torch

x = torch.ones(2, 2)
print(x)

y = torch.ones(2, 2, requires_grad=True)
print(y)

tensor([[1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


앞서 생성한 tensor에 `+3`이라는 연산을 수행했음

연산을 수행한 `y`에 `grad_fn`이 `<AddBackward0>`를 담고 있음

`grad_fn`은 해당 텐서가 어떤 연산을 수행했는지와 관련된 정보를 가지고 있고, 이 정보를 추후에 역전파에 사용함

In [2]:
print(f'x = {x}')

x1 = x + 3
print(f'x1 = {x1}')

print(f'y = {y}')

y1 = y + 3
print(f'y1 = {y1}')

x = tensor([[1., 1.],
        [1., 1.]])
x1 = tensor([[4., 4.],
        [4., 4.]])
y = tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
y1 = tensor([[4., 4.],
        [4., 4.]], grad_fn=<AddBackward0>)


사칙연산을 수행한 후에 역전파에 활용할 정보를 저장하는 `grad_fn`을 보면 각각

`AddBackward0`, `SubBackward0`, `MulBackward0`, `DivBackward0` 등이 저장되어 있음

In [6]:
x = torch.ones(2, 2, requires_grad=True)
print(f'x = {x}')
x1 = x + 2
print(f'x1 = {x1}')
x2 = x - 4
print(f'x2 = {x2}')
x3 = x * 1.2
print(f'x3 = {x3}')
x4 = x / 0.5
print(f'x4 = {x4}')

x = tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
x1 = tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
x2 = tensor([[-3., -3.],
        [-3., -3.]], grad_fn=<SubBackward0>)
x3 = tensor([[1.2000, 1.2000],
        [1.2000, 1.2000]], grad_fn=<MulBackward0>)
x4 = tensor([[2., 2.],
        [2., 2.]], grad_fn=<DivBackward0>)


원래 `reqiures_grad=False`로 default가 되어있는 tensor에 속성을 추가할 때는

`x.requires_grad_(True)` 로 앞으로는 연산 정보를 저장하게 설정해줄 수 있음

하지만, 그 전에 x가 거쳐왔던 연산들은 저장을 하고 있지 않았기 때문에

그 전의 연산들은 없는 상태로 인식됨

In [7]:
x = torch.ones(2, 2)
x1 = x + 3
x2 = x1 * 0.5

print(f'x = {x}')
print(f'x1 = {x1}')
print(f'x2 = {x2}')

x1.requires_grad_(True)
x2.requires_grad_(True)

print(f'x1 = {x1}')
print(f'x2 = {x2}')

x = tensor([[1., 1.],
        [1., 1.]])
x1 = tensor([[4., 4.],
        [4., 4.]])
x2 = tensor([[2., 2.],
        [2., 2.]])
x1 = tensor([[4., 4.],
        [4., 4.]], requires_grad=True)
x2 = tensor([[2., 2.],
        [2., 2.]], requires_grad=True)


#### Derivative

다음과 같은 식에 대해 미분을 한다고 가정.

$f(x) = 9x^4 + 2x^3 + 3x^2 + 6x + 1$

$\cfrac {df(x)}{dx} = \cfrac {d(9x^4 + 2x^3 + 3x^2 + 6x + 1)}{dx} = 36x^3 + 6x^2 + 6x + 6$

만약 $x=2$이면 derivative는 $36 \times 2^3 + 6 \times 2^2 + 6 \times 2 + 6 = 330$

위에서 말했던 대로 backward를 수행하는 y는 스칼라값이여야 함.

In [8]:
x = torch.tensor(2.0, requires_grad=True)
y = 9*x**4 + 2*x**3 + 3*x**2 + 6*x + 1
y.backward()
print(x.grad)

tensor(330.)


Matrix에 대해서 gradient를 구하고 싶다면?

1) 연산 자체에 `.sum()` 또는 `.mean()`을 사용

In [9]:
x = torch.randn(2, 2, requires_grad=True)

y = x + 3
z = (y * y).sum()
z.backward()

print(f'x = {x}')
print(f'y = {y}')
print(f'z = {z}')

print(f'x grad = {x.grad}')
print(f'y grad = {y.grad}')
print(f'z grad = {z.grad}')

x = tensor([[-0.3201,  0.0241],
        [-1.7517, -0.6088]], requires_grad=True)
y = tensor([[2.6799, 3.0241],
        [1.2483, 2.3912]], grad_fn=<AddBackward0>)
z = 23.60271644592285
x grad = tensor([[5.3598, 6.0482],
        [2.4966, 4.7823]])
y grad = None
z grad = None


/home/hwan/anaconda3/envs/torch1.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  if sys.path[0] == '':
/home/hwan/anaconda3/envs/torch1.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull

위에서 볼 수 있다시피 `y, z`에는 실제 grad가 저장되지는 않음

2) 또는 matrix의 `.backward()` 인자를 넣어주면 됨

- `torch.Tensor.backward(gradient=None)`: Computes the gradient of current tensor w.r.t. graph leaves

In [10]:
x = torch.randn(2, 2, requires_grad=True)

y = x + 2

z = (y * y)

y.backward(z)

print(x.grad)

tensor([[8.3303, 0.0103],
        [1.1132, 3.2378]])


#### Partial Derivative

$Q = 3a^3 - b^2$

1) Q는 벡터이므로 `Q.backward()`에 `gradient` 인자를 전달해야한다

2) 또는 Q.sum().backward()와 같이 Q를 scalar값으로 `aggregate`한 뒤 암시적으로 `backward()`를 호출


In [13]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 + 2 * b**2

external_grad = torch.tensor([1., 1.])
Q.backward(external_grad)

print(a.grad, b.grad)

tensor([36., 81.]) tensor([24., 16.])


In [14]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 + 2 * b**2

Q.sum().backward()
print(a.grad, b.grad)

tensor([36., 81.]) tensor([24., 16.])


#### Computational Graph
- 개념적으로 `torch.autograd`는 어떤 tensor에 실행된 모든 연산들의 기록을 `Function` 객체로 구성된 DAG(Directed Acyclic Graph)에 저장한다.

- DAG의 잎(leave)는 입력 tensor 이고, 뿌리(root)는 결과 tensor 임

- Forward에서 `torch.autograd`가 하는 일
    - 요청된 연산을 수행하여 결과 텐서를 계산
    
    - DAG에 연산의 grad_fn을 유지 함

- Backward는 DAG의 root에서 `.backward()`가 호출될 때 시작함, 이 때 `torch.autograd`가 하는 일
    - 각 `.grad_fn` 으로부터 변화도를 계산

    - 각 텐서의 `.grad` 속성에 계산 결과를 쌓는다 (accumulate)

    - chain rule을 사용하여 모든 leaf 텐서들까지 propagate 수행

`torch.autograd`는 `requires_grad`라는 플래그가 `True`인 텐서들에 대해서 연산을 추적함

따라서 derivative 계산이 필요없는 텐서는 속성값을 `False`로 설정해서 DAG에서 제외함

또한 입력 텐서 중 하나라도 `requires_grad=True`이면 결과 텐서도 변화도를 갖게됨

In [16]:
x = torch.randn(5, 5)
y = torch.randn(5, 5)
z = torch.randn(5, 5, requires_grad=True)

a = x + y
b = x + z

print(a.requires_grad)
print(b.requires_grad)

False
True


Finetuning 할 때 모델의 파라미터를 고정 시키기 (파라미터들의 변화도를 필요없게 설정하여 DAG에서 제외시키므로서 `autograd`의 연산량을 줄임)

In [17]:
import torch.nn as nn 
import torch.optim as optim
import torchvision

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/hwan/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [18]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

Linear Probing을 위해 모델의 마지막 layer만 바꾼다고 가정

In [19]:
model.fc = nn.Linear(512, 10)

optimizer = optim.SGD(model.parameters(), lr=1e-4)

Optimizer가 model의 모든 파라미터를 받지만, derivative를 계산하고, 파라미터를 update할 수 있는 부분은 fc 부분임

#### Training, Eval

`with torch.no_grad():`에서 x가 어떤 연산을 수행했을 때, `requires_grad = False`가 되는 것을 알 수 있음

- `torch.no_grad()` -- Context-manager that disabled gradient calculations
    - `torch.autograd`의 엔진이 꺼져버린다. = 앞으로 gradient의 자동 tracking을 하지 않겠다
    
    - 어차피 `loss.backward()`를 호출 안하고, backpropagation도 안하는데 굳이 gradient 계산을 안하게 하는 이유?

    - 메모리 사용량을 줄이고 연산 속도를 높히려고

- `model.eval()` -- Batch normalization, dropout과 같은 것들을 evaluation mode로 바꿔줌

In [22]:
x = torch.tensor(1.0, requires_grad=True)

print(x.requires_grad)

with torch.no_grad():
    print(x.requires_grad)
    print((x + 3).requires_grad)

print(x.requires_grad)

True
True
False
True


### 여러가지 유용한 함수들
- `torch.gather(input, dim, index)` : Gathers values along an axis specified by `dim`

- `torch.tensor.gather(dim, index)`

- `torch.tensor.expand(sizes)` : Returns a new view of the `self` tensor with singleton dimensions expanded to a larger size

- `torch.tensor.repeat(sizes)` : Repeats this tensor along the specified dimensions / Unlike `expand()`, this function copies the tensor's data

- `torch.topk(input, k, dim=None, largest=True, sorted=True)` : Returns the k largest elements of the given `input` tensor along a given dimension

In [23]:
# torch.gather 은 index를 기준으로 tensor의 특정 값을 추출하기 위해 사용
# 파이썬 내장 indexing 기능과 유사하지만, 최적화나 호환성을 위해 torch 함수를 최대한 사용해야 하는 상황에 torch.gather 사용

x = torch.arange(1, 10)
print(x)

indices = torch.tensor([0, 3, 5, 6])
print(indices)

torch.gather(x, dim=0, index=indices)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([0, 3, 5, 6])


tensor([1, 4, 6, 7])

In [26]:
# 이차원인 경우에 indices도 똑같이 생각하면 편리함
# indices의 0번째 행의 값 = tensor의 0번째 행에서 가져올 값들의 인덱스
# 이때 gather에 들어갈 tensor의 

x = torch.arange(25).reshape(5, 5)
print(x)

indices = torch.tensor([
    [0, 1, 2],
    [1, 2, 3],
    [1, 2, 2],
    [2, 3, 3],
    [0, 0, 0]
])

print(torch.gather(x, 1, indices))

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]])
tensor([[ 0,  1,  2],
        [ 6,  7,  8],
        [11, 12, 12],
        [17, 18, 18],
        [20, 20, 20]])


In [27]:
# torch.expand 와 torch.repeat 모두 값을 반복시키는 연산
# torch.expand는 특정 tensor를 반복하며 생성하고, 개수가 1인 차원에만 적용 가능

x = torch.tensor([[1], [2], [3]])
print(x.size())

x1 = x.expand(3, 4) # 3 x 1짜리를 3 x 4로 늘리니깐 복사
print(x1, x1.size())

x2 = x.expand(-1, 4) # 차원이 1인 곳만 expand해서 증가하기 때문에 1을 4로 늘리고 나머지는 알아서 계산하라고 -1
print(x2, x2.size())

torch.Size([3, 1])
tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]]) torch.Size([3, 4])
tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]]) torch.Size([3, 4])


In [28]:
x = torch.rand(3, 1, 1)
print(x, x.size())

z = x.expand(-1, 2, 4)
print(z, z.size())

tensor([[[0.2199]],

        [[0.7933]],

        [[0.7867]]]) torch.Size([3, 1, 1])
tensor([[[0.2199, 0.2199, 0.2199, 0.2199],
         [0.2199, 0.2199, 0.2199, 0.2199]],

        [[0.7933, 0.7933, 0.7933, 0.7933],
         [0.7933, 0.7933, 0.7933, 0.7933]],

        [[0.7867, 0.7867, 0.7867, 0.7867],
         [0.7867, 0.7867, 0.7867, 0.7867]]]) torch.Size([3, 2, 4])


In [29]:
# torch.expand가 차원이 1인 값을 원하는 크기만큼 복사에서 늘리는 거라면
# torch.repeat는 tensor 자체를 반복해서 값을 채운다

x = torch.rand(2, 3)
print(x, x.size())

y = x.repeat(3, 2, 2)
print(y, y.size()) # x는 2 x 3인데 이에 3 x 2 x 2 만큼 반복을 하면 (3, 2 x 2, 2 x 3) = (3, 4, 6)이 된다

tensor([[0.6951, 0.6928, 0.7578],
        [0.6913, 0.5426, 0.1170]]) torch.Size([2, 3])
tensor([[[0.6951, 0.6928, 0.7578, 0.6951, 0.6928, 0.7578],
         [0.6913, 0.5426, 0.1170, 0.6913, 0.5426, 0.1170],
         [0.6951, 0.6928, 0.7578, 0.6951, 0.6928, 0.7578],
         [0.6913, 0.5426, 0.1170, 0.6913, 0.5426, 0.1170]],

        [[0.6951, 0.6928, 0.7578, 0.6951, 0.6928, 0.7578],
         [0.6913, 0.5426, 0.1170, 0.6913, 0.5426, 0.1170],
         [0.6951, 0.6928, 0.7578, 0.6951, 0.6928, 0.7578],
         [0.6913, 0.5426, 0.1170, 0.6913, 0.5426, 0.1170]],

        [[0.6951, 0.6928, 0.7578, 0.6951, 0.6928, 0.7578],
         [0.6913, 0.5426, 0.1170, 0.6913, 0.5426, 0.1170],
         [0.6951, 0.6928, 0.7578, 0.6951, 0.6928, 0.7578],
         [0.6913, 0.5426, 0.1170, 0.6913, 0.5426, 0.1170]]]) torch.Size([3, 4, 6])


In [33]:
x = torch.rand(2, 3, 2)

# (2 x 3 x 2) repeat (3 x 4 x 5 x 6) = (3, 2 x 4, 3 x 5, 2 x 6) = (3, 8, 15, 12)

x.repeat(3, 4, 5, 6).size()

torch.Size([3, 8, 15, 12])

In [34]:
### torch.expand와 torch.repeat는 내부적으로 동작방식이 다름
### torch.expand는 원본 tensor를 참조하여 만들기 때문에 원래 tensor가 변경되면 expand된 값도 변경됨
### 반면에 torch.repeat는 deepcopy에 의해 생성되기 때문에 원래 tensor가 변경되어도 변하지 않음

x = torch.rand(2, 3, 1)
print(x)
x_expand = x.expand(2, 3, 5)
print(x_expand)

x += 1
print(x)
print(x_expand)

tensor([[[0.5299],
         [0.2750],
         [0.5214]],

        [[0.6904],
         [0.3287],
         [0.7033]]])
tensor([[[0.5299, 0.5299, 0.5299, 0.5299, 0.5299],
         [0.2750, 0.2750, 0.2750, 0.2750, 0.2750],
         [0.5214, 0.5214, 0.5214, 0.5214, 0.5214]],

        [[0.6904, 0.6904, 0.6904, 0.6904, 0.6904],
         [0.3287, 0.3287, 0.3287, 0.3287, 0.3287],
         [0.7033, 0.7033, 0.7033, 0.7033, 0.7033]]])
tensor([[[1.5299],
         [1.2750],
         [1.5214]],

        [[1.6904],
         [1.3287],
         [1.7033]]])
tensor([[[1.5299, 1.5299, 1.5299, 1.5299, 1.5299],
         [1.2750, 1.2750, 1.2750, 1.2750, 1.2750],
         [1.5214, 1.5214, 1.5214, 1.5214, 1.5214]],

        [[1.6904, 1.6904, 1.6904, 1.6904, 1.6904],
         [1.3287, 1.3287, 1.3287, 1.3287, 1.3287],
         [1.7033, 1.7033, 1.7033, 1.7033, 1.7033]]])


In [35]:
x = torch.randn(1, 2)

print(x)

y = x.repeat(2, 1, 2)

print(y)

x[0] = 0
print(x)

print(y)

tensor([[-0.6371, -1.3024]])
tensor([[[-0.6371, -1.3024, -0.6371, -1.3024]],

        [[-0.6371, -1.3024, -0.6371, -1.3024]]])
tensor([[0., 0.]])
tensor([[[-0.6371, -1.3024, -0.6371, -1.3024]],

        [[-0.6371, -1.3024, -0.6371, -1.3024]]])


In [36]:
# torch.topk는 tensor에서 가장 큰 k 개의 값을 추출하는 연산이다.
# torch.topk의 output은 tuple의 형태로 (values: Tensor, indices: LongTensor)로 나옴

x = torch.arange(1, 6)
print(x)

values, indices = torch.topk(x, 3)
print(values)
print(indices)

tensor([1, 2, 3, 4, 5])
tensor([5, 4, 3])
tensor([4, 3, 2])


In [38]:
x = torch.rand(2, 4, 3)
print(x)

val, idx = torch.topk(x, 2)
print(val)
print(idx)

tensor([[[0.4066, 0.4182, 0.5242],
         [0.0529, 0.8201, 0.9994],
         [0.0452, 0.0121, 0.8284],
         [0.1758, 0.5043, 0.1529]],

        [[0.5257, 0.3002, 0.0814],
         [0.4746, 0.7907, 0.8573],
         [0.5949, 0.4687, 0.9740],
         [0.4998, 0.7100, 0.7023]]])
tensor([[[0.5242, 0.4182],
         [0.9994, 0.8201],
         [0.8284, 0.0452],
         [0.5043, 0.1758]],

        [[0.5257, 0.3002],
         [0.8573, 0.7907],
         [0.9740, 0.5949],
         [0.7100, 0.7023]]])
tensor([[[2, 1],
         [2, 1],
         [2, 0],
         [1, 0]],

        [[0, 1],
         [2, 1],
         [2, 0],
         [1, 2]]])


In [40]:
# topk의 indices와 torch.gather을 같이 사용할 수 있음

torch.gather(x, -1, idx) == val

tensor([[[True, True],
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         [True, True]]])

In [42]:
# 둘이 같이 쓰는 경우
# x와 y의 shape이 같은 상황에서, x의 최댓값이 있는 위치의 y는 어떤 값을 갖는지 궁금하다

x = torch.rand(3, 4, 2)
y = torch.rand(3, 4, 2)

val, idx = torch.topk(x, 1, dim=2)

torch.gather(y, 2, idx)

tensor([[[0.1852],
         [0.5325],
         [0.3681],
         [0.1506]],

        [[0.6125],
         [0.8904],
         [0.9535],
         [0.4875]],

        [[0.9080],
         [0.9120],
         [0.4572],
         [0.1577]]])